In [1]:
pip install selenium

In [3]:
pip install undetected-chromedriver

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install undetected-chromedriver pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:

import time
import random
import os
import pandas as pd
import undetected_chromedriver as uc
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# --- CONFIGURATION ---

LISTE_URLS = [
    "https://fr.aliexpress.com/item/1005010590742420.html",
    "https://fr.aliexpress.com/item/1005010555389635.html",
    "https://fr.aliexpress.com/item/1005004508111042.html",
    "https://fr.aliexpress.com/item/1005010192224932.html",
    "https://fr.aliexpress.com/item/1005010590280107.html",
    "https://fr.aliexpress.com/item/1005009150347286.html",
    "https://fr.aliexpress.com/item/1005009242077977.html",
    "https://fr.aliexpress.com/item/1005010629062078.html",
    "https://fr.aliexpress.com/item/1005010363351298.html",
    "https://fr.aliexpress.com/item/1005009255030706.html",
    "https://fr.aliexpress.com/item/1005006220028020.html",
    "https://fr.aliexpress.com/item/1005010787177653.html",
    "https://fr.aliexpress.com/item/1005010711558780.html",
    "https://fr.aliexpress.com/item/1005010601871827.html",
    "https://fr.aliexpress.com/item/1005010590123600.html",
    "https://fr.aliexpress.com/item/1005010711842135.html",
    "https://fr.aliexpress.com/item/1005006116671134.html",
    "https://fr.aliexpress.com/item/1005006770373330.html",
    "https://fr.aliexpress.com/item/1005003087229428.html",
    "https://fr.aliexpress.com/item/1005010187143364.html",
    "https://fr.aliexpress.com/item/1005005852421208.html",
    "https://fr.aliexpress.com/item/1005008675678421.html",
    "https://fr.aliexpress.com/item/1005006295167658.html",
    "https://fr.aliexpress.com/item/1005004274694269.html",
    "https://fr.aliexpress.com/item/1005005353294268.html",
    "https://fr.aliexpress.com/item/32847849368.html",
    "https://fr.aliexpress.com/item/1005010362582982.html",
    "https://fr.aliexpress.com/item/1005011559963709.html",
    "https://fr.aliexpress.com/item/1005004220019939.html",
    "https://fr.aliexpress.com/item/1005006220105307.html"
]

EXCEL_FILE = "suivi_complet_aliexpress.xlsx"

# --- FONCTIONS TECHNIQUES ---

def get_driver():
    """Initialise le navigateur avec des options de furtivité renforcées."""
    options = uc.ChromeOptions()
    
    # METHODE ANTI-BOT 1 : Taille de fenêtre aléatoire
    # Les bots utilisent souvent 1920x1080. Varier simule différents écrans.
    w = random.randint(1200, 1920)
    h = random.randint(800, 1080)
    options.add_argument(f"--window-size={w},{h}")
    
    # METHODE ANTI-BOT 2 : Désactivation du WebGL et des images
    # Réduit la charge et cache certaines empreintes matérielles.
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--blink-settings=imagesEnabled=false")
    
    # Initialisation du driver furtif
    driver = uc.Chrome(options=options, version_main=143, use_subprocess=True)
    
    # METHODE ANTI-BOT 3 : Injection de scripts pour modifier l'empreinte JS
    # On supprime la mention "webdriver" dans les propriétés du navigateur
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
        """
    })
    
    return driver

def force_close_popups(driver):
    """Gère les cookies et les pop-ups qui trahissent souvent les bots par leur non-réponse."""
    try:
        wait = WebDriverWait(driver, 5)
        cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, 
            "//button[contains(.,'Accepter') or contains(.,'Accept') or contains(.,'Okay')]")))
        # METHODE ANTI-BOT 4 : Pause humaine avant de cliquer
        time.sleep(random.uniform(0.5, 1.5))
        cookie_btn.click()
    except:
        pass

def scrape_product(driver, url):
    """Navigue et extrait les informations avec simulation humaine."""
    try:
        driver.get(url)
        
        # METHODE ANTI-BOT 5 : Temps de lecture aléatoire (Dwell Time)
        # Un humain ne charge pas 30 pages en 30 secondes.
        time.sleep(random.uniform(8, 15))
        
        force_close_popups(driver)
        
        # METHODE ANTI-BOT 6 : Mouvements de souris aléatoires
        # Simule un utilisateur qui déplace sa souris avant d'extraire
        action = ActionChains(driver)
        action.move_by_offset(random.randint(10, 100), random.randint(10, 100)).perform()
        
        now = datetime.now()
        data = {
            "date": now.strftime("%d/%m/%Y"),
            "heure": now.strftime("%H:%M:%S"),
            "nom_article": "N/A",
            "prix": "N/A",
            "vendeur_nom": "N/A",
            "vendeur_location": "N/A",
            "vendeur_date_ouverture": "N/A",
            "url": url
        }

        try:
            full_title = driver.find_element(By.XPATH, "//h1").text.strip()
            data["nom_article"] = full_title
            data["prix"] = driver.find_element(By.CSS_SELECTOR, "span.price-default--current--F8OlYIo").text.strip()
        except: 
            pass

        try:
            # METHODE ANTI-BOT 7 : Scroll par étapes (Smooth Scrolling)
            # Les bots sautent directement en bas, les humains défilent.
            for i in range(3):
                driver.execute_script(f"window.scrollBy(0, {random.randint(100, 250)});")
                time.sleep(random.uniform(0.3, 0.7))

            hover_zone = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "store-detail--title--qt8UBeq")))
            
            # Hover réaliste
            actions = ActionChains(driver)
            actions.move_to_element(hover_zone).pause(random.uniform(0.5, 1.0)).perform()
            time.sleep(3)

            def get_val(lbl):
                try: 
                    return driver.find_element(By.XPATH, f"//td[contains(text(), '{lbl}')]/following-sibling::td").text.strip()
                except: 
                    return "N/A"

            data["vendeur_nom"] = get_val("Nom") or get_val("Name")
            data["vendeur_location"] = get_val("Emplacement") or get_val("Location")
            data["vendeur_date_ouverture"] = get_val("ouverture") or get_val("Open Date")
        except: 
            pass

        return data
    except Exception as e:
        print(f"Erreur sur {url} : {e}")
        return None

def save_to_excel(new_data_list):
    """Enregistre les données sans perte d'historique."""
    df_new = pd.DataFrame(new_data_list)
    if os.path.exists(EXCEL_FILE):
        df_old = pd.read_excel(EXCEL_FILE)
        df_final = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_final = df_new
    df_final.to_excel(EXCEL_FILE, index=False)
    return df_final

# --- BOUCLE DE SURVEILLANCE PRINCIPALE ---

print(f"Tracker active a {datetime.now().strftime('%H:%M:%S')}. (Cycle de 30 min)")

while True:
    session_start = datetime.now()
    print(f"\n--- SESSION DU {session_start.strftime('%d/%m/%Y a %H:%M:%S')} ---")
    
    # METHODE ANTI-BOT 8 : Nouvelle instance à chaque session
    # On ferme et réouvre Chrome pour vider le cache et les cookies de session.
    driver = get_driver()
    session_results = []

    try:
        # METHODE ANTI-BOT 9 : Ordre de visite aléatoire
        # Ne pas visiter les URLs toujours dans le même ordre.
        urls_random = LISTE_URLS.copy()
        random.shuffle(urls_random)

        for i, url in enumerate(urls_random):
            res = scrape_product(driver, url)
            if res:
                session_results.append(res)
                nom_court = res['nom_article'][:30] + "..." if len(res['nom_article']) > 30 else res['nom_article']
                print(f"   [{i+1}/{len(LISTE_URLS)}] {res['prix']} | {nom_court} | {res['vendeur_nom']}")
            
            # Pause entre chaque produit
            time.sleep(random.uniform(5, 12))
            
            # METHODE ANTI-BOT 10 : Pause "Micro-sommeil" longue
            # Toutes les quelques requêtes, on s'arrête plus longtemps.
            if (i + 1) % random.randint(7, 12) == 0:
                print("   Pause respiration longue (90s) pour eviter le blocage IP...")
                time.sleep(random.uniform(80, 110))

        if session_results:
            save_to_excel(session_results)
            print("\nRECAPITULATIF FINAL DE LA SESSION SAUVEGARDE")

    except Exception as e:
        print(f"Erreur durant la session : {e}")
    finally:
        driver.quit()

    # Calcul du temps d'attente pour le prochain cycle de 30 min
    elapsed = (datetime.now() - session_start).total_seconds()
    # On ajoute une variation aléatoire au temps d'attente global (30 min +/- 5 min)
    wait_time = max(0, (1800 + random.randint(-300, 300)) - elapsed)
    
    print(f"\nTermine. Prochaine verification a : {datetime.fromtimestamp(time.time() + wait_time).strftime('%H:%M:%S')}")
    time.sleep(wait_time)

Tracker active a 18:12:38. (Cycle de 30 min)

--- SESSION DU 02/02/2026 a 18:12:38 ---
   [1/30] 38,59€ | Téléphone portable d'origine i... | BanMiDi Original Brand Phone Store Store
   [2/30] 135,99€ | Original Apple iPhone 8 Plus 8... | PhoneArena Online Store
   [3/30] 63,99€ | iPhone 6S 6SPLUS débloqué 2GB ... | Hong Kong Brother United Group Technology Co.,Limited Store
   [4/30] 158,99€ | Apple iPhone XS 5.8 "/XS MAX S... | Rorgoo Store
   [5/30] 72,39€ | Original débloqué Apple iPhone... | BanMiDi Original Brand Phone Store Store
   [6/30] N/A | N/A | N/A
   [7/30] N/A | N/A | N/A
   Pause respiration longue (90s) pour eviter le blocage IP...
   [8/30] N/A | N/A | N/A
Erreur sur https://fr.aliexpress.com/item/1005009242077977.html : Message: move target out of bounds
  (Session info: chrome=144.0.7559.110)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xa11213
	0xa11254
	0x7fe52b
	0x895f21
	0x86b4ec
	0x88db13
	0x86b2e6
	0x83d321
	0x83e1d4
	0xc65254
	0xc6080b
